In [4]:
!pip install corus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 2.0 MB/s eta 0:00:00


In [5]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-s0e3uqz1
  Running command git clone --filter=blob:none --quiet https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-s0e3uqz1
  Resolved https://github.com/lopuhin/python-adagram.git to commit cf3639f10d6a1efbcb602f45a1da89ef55ce5794
  Preparing metadata (setup.py) ... done
  Created wheel for adagram: filename=adagram-0.0.1-cp310-cp310-linux_x86_64.whl size=686337 sha256=f0324a71c553ab9431e780ba24cbdd69fc21f1feafe3400b362281174b535dec
  Stored in directory: /tmp/pip-ephem-wheel-cache-309azf_q/wheels/88/8b/fc/291709600177e98c9bba397433476b87d894ca506e78302432
Successfully built adagram


In [6]:
!pip install pymystem3 pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 58.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=789df3fa72bcbb09bda71ccb1a1eb6eab20ac0078e50e5128ea1c41cfd0032bd
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [7]:
import re
from corus import load_lenta
from nltk.tokenize import sent_tokenize
from pymorphy2 import MorphAnalyzer
import pandas as pd
import adagram
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem
import numpy as np
import nltk
nltk.download('punkt')
import pandas as pd
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')


def tokenize(text):
    return token.tokenize(text)

In [9]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  51.3M      0  0:00:27  0:00:27 --:--:-- 47.2M


In [10]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

1. Слова были собраны вручную с использованием МАС

In [32]:
wed = []
det = []
pre = []
smll = []
dsh = []

2. Из коруса Lenta.ru v1.0 коллекции Corus были извлечены предложения с выбранными неоднозначными словами (*среда, деталь, предлог, мелочь, блюдо*)

In [33]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [35]:
for record in records:
    txt = record.text
    patt_w = r'\bсред([ауеы][^а-яА-Я]|[^а-яА-Я]|ах|ами|ой|ам|ою)'
    patt_de = r'\bдетал([ьи][^а-яА-Я]|[^а-яА-Я]|ях|ью|ей|ями|ям)'
    patt_p = r'\bпредлог([ауеи][^а-яА-Я]|[^а-яА-Я]|ом|ов|ами|ам|ах)'
    patt_s = r'\bмелоч([ьи][^а-яА-Я]|[^а-яА-Я]|ью|ей|ами|ам|ах)'
    patt_di = r'\bблюд([оауе][^а-яА-Я]|[^а-яА-Я]|ом|ами|ам|ах)'
    for s in sent_tokenize(txt):
        if re.search(patt_w, s.lower()):
            wed.append(s)
        if re.search(patt_de, s.lower()):
            det.append(s)
        if re.search(patt_p, s.lower()):
            pre.append(s)
        if re.search(patt_s, s.lower()):
            smll.append(s)
        if re.search(patt_di, s.lower()):
            dsh.append(s)

In [36]:
data = []
for i in wed:
  data.append(['среда', i])
for i in det:
  data.append(['деталь', i])
for i in pre:
  data.append(['предлог', i])
for i in smll:
  data.append(['мелочь', i])
for i in dsh:
  data.append(['блюдо', i])

In [37]:
pd.set_option('display.max_colwidth', 1000)

In [38]:
df = pd.DataFrame(data, columns=['lemma', 'text'])

3. Полученные контексты распределялись по значениям с помощью AdaGram

In [39]:
mystem = Mystem()

def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()

In [40]:
df['predict_sense'] = df.apply(lambda x: disambiguate(vm, x['lemma'], x['text']), axis=1)

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [41]:
df.head(2)

,lemma,text,predict_sense
0,среда,"Жена президента США Дональда Трампа Мелания Трамп появилась на публике в среду, 12 декабря, с новым цветом волос, который рассмешил зрителей.",2
1,среда,"Инцидент произошел в одной из деревень муниципалитета Богор, провинция Западная Ява, в среду, 12 декабря.",2


4. Словарные толкования также распределялись по значению с помощью AdaGram

In [42]:
word_senses = [{'lemma': 'среда', 'sense': 'Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами.'},
        {'lemma': 'среда', 'sense': 'Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма.'},
        {'lemma': 'среда', 'sense': 'Социально-бытовая обстановка, в которой протекает жизнь человека; окружение.'},
        {'lemma': 'среда', 'sense': 'Третий (после воскресенья) день недели (середина недели).'},
        {'lemma': 'деталь', 'sense': 'Мелкая подробность, частность.'},
        {'lemma': 'деталь', 'sense': 'Часть механизма, машины, которая не может быть разобрана на другие, более простые и мелкие.'},
        {'lemma': 'предлог', 'sense': 'Повод к чему-л., вымышленная причина.'},
        {'lemma': 'предлог', 'sense': 'Служебное слово (например: в, к, на, при), которое, сочетаясь с существительными, местоимениями и числительными, указывает на синтаксические отношения их к другим словам.'},
        {'lemma': 'мелочь', 'sense': 'Предметы небольшой величины; мелкие вещи.'},
        {'lemma': 'мелочь', 'sense': 'Предметы или живые существа менее крупные по сравнению с другими однородными.'},
        {'lemma': 'мелочь', 'sense': 'Мелкие деньги.'},
        {'lemma': 'мелочь', 'sense': 'Что-л. не имеющее существенного значения; пустяк.'},
        {'lemma': 'блюдо', 'sense': 'Столовая посуда в виде большой тарелки, круглой или овальной формы, для подачи к столу кушанья.'},
        {'lemma': 'блюдо', 'sense': 'Определенным образом приготовленные для еды продукты питания; кушанье.'}]
df_senses = pd.DataFrame(word_senses)


In [44]:
df_senses['predict_sense'] = df_senses.apply(lambda x: disambiguate(vm, x['lemma'], x['sense']), axis=1)
df_senses

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


,lemma,sense,predict_sense
0,среда,"Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами.",3
1,среда,"Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма.",3
2,среда,"Социально-бытовая обстановка, в которой протекает жизнь человека; окружение.",4
3,среда,Третий (после воскресенья) день недели (середина недели).,2
4,деталь,"Мелкая подробность, частность.",3
5,деталь,"Часть механизма, машины, которая не может быть разобрана на другие, более простые и мелкие.",1
6,предлог,"Повод к чему-л., вымышленная причина.",2
7,предлог,"Служебное слово (например: в, к, на, при), которое, сочетаясь с существительными, местоимениями и числительными, указывает на синтаксические отношения их к другим словам.",2
8,мелочь,Предметы небольшой величины; мелкие вещи.,2
9,мелочь,Предметы или живые существа менее крупные по сравнению с другими однородными.,5


5. Контексты размечались вручную и собирались в файл gold_sense.tsv (контексты со словом предлог не размечались, поскольку AdaGram посчитал его толкования одним значением)

In [42]:
# df.to_csv('corp.tsv', sep='\t', index=False, header=True)

In [46]:
compr_df = pd.read_csv('gold_sense.tsv', sep='\t')
compr_df.head()

,lemma,text,predict_sense,gold_sense
0,среда,"Жена президента США Дональда Трампа Мелания Трамп появилась на публике в среду, 12 декабря, с новым цветом волос, который рассмешил зрителей.",2,2
1,среда,"Инцидент произошел в одной из деревень муниципалитета Богор, провинция Западная Ява, в среду, 12 декабря.",2,2
2,среда,"В среду, 12 декабря, полиция подтвердила, что останки принадлежат отцу мужчины.",2,2
3,среда,"Взрыв произошел в среду, 12 декабря, около 18:00 в торговом центре на улице Картоева в Назрани.",2,2
4,среда,"Видео задержания в среду, 13 декабря, опубликовано на YouTube-канале СК.",2,2


6. Оценка accuracy

In [47]:
accur = compr_df.groupby('lemma').apply(lambda x: accuracy_score(x['gold_sense'], x['predict_sense'])).to_frame('accuracy')

In [48]:
accur

,accuracy
lemma,
блюдо,0.600000
деталь,1.000000
мелочь,0.777778
среда,0.800000
